# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante 

### SAPORI SOLARI - Milano

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187849-d8780418-Reviews-Sapori_Solari-Milan_Lombardy.html

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187849-d8780418-Reviews-Sapori_Solari-Milan_Lombardy.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187895-d15272936-Reviews-or10-SENZ_ALTRO_Bistrot-Florence_Tuscany.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187895-d15272936-Reviews-or'
URL_p2 = '0-SENZ_ALTRO_Bistrot-Florence_Tuscany.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in range(1,total_pages):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
total_pages:  112
Nomi:  1098
Commenti:  1098
Voti:  1098
Scritto da:  1098


In [0]:
# display(all_reviews)

In [0]:
# print(all_names)

In [0]:
# print(all_ratings)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [8]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate)

display(all_reviews_filtered)

Totale recensioni: 1098 
 Recensioni analizzabili: 933 
 Recensioni scartate: 165


['RECENSIONE_TROPPO_LUNGA',
 'Ambiente accogliente, familiare. I prodotti sono di ottima qualità e spiegati con grande passione. Consigliatissimo per chi vuole provare qualcosa di nuovo 😊',
 'idea, proposta e impegno dei ragazzi ok ma ho assaggiato il tagliere di mare e non ci siamo..troppo sale e troppo fumo, decisamente no. Peccato..',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'Ottimi salumi e formaggi presentati con professionalità da un cortese oste. Vini apprezzabili prezzo equo per un aperitivo',
 'Personale gentile e professionale, prodotti eccezionali, buon vino e ottima qualità-prezzo.\nConsiglio vivamente!!! 😊',
 'RECENSIONE_TROPPO_LUNGA',
 'Una cena raccontata e degustata con molta lentezza e ascolto dei sapori così finemente accostati. Se pensate ad una cena veloce ... non andateci! Lo staff intrattiene, spiega, racconta e accompagna i commensali alla scoperta di nuovi sapori.\nUn posto diverso... che fa la differenza',
 'Sempre una scoperta emozionante prova

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [9]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)          # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['supermariocast', 'Alice9955', 'MikiMouse84', 'Giampaolo m', 'Melissa B', 'Patrizia1557', 'Francesca D', 'AlfonsOlivieri', 'Finis5terrae', 'Inspire261783', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'Angela S', 'Erika P', 'PattyPatrizia', 'alessandra z', 'Sabrina P', 'Simona L', 'ilmatto94', 'giuseppecT8820YG', 'Antonio T', 'deniserapisardaz', 'gngiulianocentini', 'Rualb77', 'francescoN414', 'Z9570TCluigic', 'Francesca C', 'Alfio C', 'Melina L', 'Mariagrazia L', '122filippof', 'LoveTravel687476', 'giorgiof798', 'loresupermoto', 'Patrizia P', 'Giusi B', 'Carmen B', 'bloomblo', 'Cekkky', 'Andrea T', 'Ocram P', 'camaleonte2015', 'Salvatore G', 'veronica f', 'Gabriel B', 'eleonoram0203', 'Lisina89', 'francesco l', 'Gianluca P', 'Alex1966Juve', 'serenaa103', 'danielev814', 'GiulRoss', 'Nikola I', 'Carmela C', 'nicolevispi', 'Gabriele S', 'Dino F', 'Freedom582758', 'matteogG5456HG', 'alitash', 'Noemi G', 'Rachele_tudisco', '

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [10]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'female', 'unknown', 'male', 'female', 'female', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'female', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'female', 'female', 'female', 'female', 'male', 'unknown', 'female', 'mostly_female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown

In [11]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'F', 'UNKNOWN', 'M', 'F', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'F', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'F', 'F', 'F', 'F', 'M', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'M', 'M', 'UNKNOWN', 'F', 'M', 'M', 'F', 'UNKNOWN', 'UNKNOW

---
### Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [12]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  1098
Commenti:  1098
Voti:  1098
Mobile:  1098


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [13]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  228


[['Alice',
  'F',
  'Ambiente accogliente, familiare. I prodotti sono di ottima qualità e spiegati con grande passione. Consigliatissimo per chi vuole provare qualcosa di nuovo 😊',
  '5',
  'Mobile'],
 ['Patrizia',
  'F',
  'Ottimi salumi e formaggi presentati con professionalità da un cortese oste. Vini apprezzabili prezzo equo per un aperitivo',
  '4',
  'PC'],
 ['Francesca',
  'F',
  'Personale gentile e professionale, prodotti eccezionali, buon vino e ottima qualità-prezzo.\nConsiglio vivamente!!! 😊',
  '5',
  'Mobile'],
 ['Angela',
  'F',
  "Bellissimo bistrot... pranzo ottimo, maialino squisito e vino speciale.Il servizio e come sempre impeccabile e l'ambiente accogliente... ritornarci è stata una conferma",
  '5',
  'Mobile'],
 ['Antonio',
  'M',
  'Mai mangiato così bene in vacanza.... Bistrot ai primissimi livelli sul panorama nazionale! Ottimo primo e secondi da leccarsi i baffi... E che dessert!',
  '5',
  'Mobile']]

### Creiamo la tabella

In [0]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [0]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_1.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [16]:
partial_dataframe = pd.read_excel("RISTORANTE_1.xlsx")

display(partial_dataframe)

,Genere,Nomi,Recensioni,Scritta da,Voti
0,F,Alice,"Ambiente accogliente, familiare. I prodotti so...",Mobile,5
1,F,Patrizia,Ottimi salumi e formaggi presentati con profes...,PC,4
2,F,Francesca,"Personale gentile e professionale, prodotti ec...",Mobile,5
3,F,Angela,"Bellissimo bistrot... pranzo ottimo, maialino ...",Mobile,5
4,M,Antonio,Mai mangiato così bene in vacanza.... Bistrot ...,Mobile,5
5,F,Francesca,Eccelente ristorante con cuochi molto esperti ...,Mobile,5
6,M,Alfio,"Ero già stato qui in precedenza, così domenica...",Mobile,5
7,F,Melina,E la 2 volta che ritorno in questo ristorante ...,Mobile,5
8,F,Patrizia,Cibo squisito preparato tutto nei minimi detta...,Mobile,5
9,F,Giusi,Leggendo le recensioni con i miei amici abbiam...,Mobile,5
